In [3]:
#importing all necesssary Libraries
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException,ElementClickInterceptedException
from selenium.webdriver.common.by import By
import time
import requests
import os
import warnings
warnings.filterwarnings('ignore')

#### 1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

#### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are:
1. Brand Name
2. Name of the Product
3. Price 
4. Return/Exchange 
5. Expected Delivery 
6. Availability 
7. Product URL

- In case, if any of the details are missing for any of the product then replace it by “-“.

In [2]:
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.amazon.in/')
time.sleep(2)

driver.maximize_window()
time.sleep(1)

Product = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
Product.send_keys('Guitar')
time.sleep(1)

search = driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search.click() 
time.sleep(6)

#Scraping the Urls of the products
product_urls = []

for i in range(3):
    url = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
    for i in url:
        product_urls.append(i.get_attribute('href'))
    next_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    time.sleep(2)

In [5]:
Name = []
Brand_Name = []
Price = []
Expected_Delivery = []
Return = []
Availability = []


for i in product_urls:
    driver.get(i)
    time.sleep(4)
    
    try:
        name = driver.find_element(By.XPATH,'//span[@id="productTitle"]')
        Name.append(name.text) if i else Name.append('-')
    except NoSuchElementException:
        Name.append('-')
    
    try: 
        brand = driver.find_element(By.XPATH,'//tr[@class="a-spacing-small po-brand"]')
        Brand_Name.append(brand.text.split(' ')[1]) if i else Brand_Name.append('-')
    except NoSuchElementException:
        Brand_Name.append('-')

    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')
        Price.append(price.text) if i else Price.append('-')
    except NoSuchElementException:
        price = driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none aok-align-center"]')
        Price.append(price.text.split('\n')[1])

    try:
        #For expected Delivery date
        expec = driver.find_element(By.XPATH,'//*[@id="mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE"]/span/span')
        Expected_Delivery.append(expec.text) if expec else Expected_Delivery.append('-')
    except NoSuchElementException:
        Expected_Delivery.append('-')
    
    try:
        #For return/Exchange
        retur = driver.find_element(By.XPATH,'//div[@id="RETURNS_POLICY"]')
        Return.append(retur.text) if retur else Return.append('-')
    except NoSuchElementException:
        Return.append('-')
        
        
    try:
        #For Availability
        avail = driver.find_element(By.XPATH,'//*[@id="availability"]/span')
        Availability.append(avail.text) if avail else Availability.append('-')     
    except NoSuchElementException:
        Availability.append('-')
        

In [6]:
len(Brand_Name),len(Price),len(Name),len(Expected_Delivery),len(Return),len(Availability)

(187, 187, 187, 187, 187, 187)

In [7]:
data = {
    'Brand':Brand_Name,
    'Name':Name,
    'Price':Price,
    'Delivery':Expected_Delivery,
    'Return/Exchange':Return,
    'Availability':Availability,
    'Product Link':product_urls
      
}

In [8]:
df = pd.DataFrame(data)
df

,Brand,Name,Price,Delivery,Return/Exchange,Availability,Product Link
0,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,949","Wednesday, 28 December",7 days Replacement,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,900","Thursday, 29 December",7 days Replacement,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Medellin,Medellin M38 carbon fiber body 38 Incheses Aco...,"2,428","Wednesday, 28 December",7 days Replacement,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Medellin,"Medellin 38"" Acoustic Guitar premium wood with...","2,398","Thursday, 29 December",7 days Returnable,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","2,149","Wednesday, 28 December",7 days Replacement,In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
182,-,Takamine GD71CE-NAT Dreadnought Cutaway Acoust...,"1,26,255",January 12 - 16,7 days Replacement,Usually dispatched in 3 to 4 days.,https://www.amazon.in/Takamine-GD71CE-NAT-Drea...
183,Techblaze,Techblaze 22 Pcs Guitar Kit Set Guitar Strings...,769,"Friday, 30 December",7 days Replacement,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
184,blueberry,"Blueberry B-40, 40"" Acoustic Guitar Kit Inbuil...","4,990","Thursday, 29 December",7 days Replacement,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
185,Gibz's,Gibz's G-38 Matte Acoustic Guitar With Truss R...,"4,498","Friday, 30 December",7 days Replacement,Only 2 left in stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [9]:
df.to_csv('AmazonProducts.csv',index=False)
driver.close()

#### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [34]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://images.google.com/') 
time.sleep(1)

# searching for cakes
keyword = driver.find_element(By.XPATH,'//input[@class="gLFyf"]').send_keys('Cakes')
time.sleep(1)
search = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]').click()
driver.maximize_window()
time.sleep(2)

for _ in range(100):
    driver.execute_script('window.scrollBy(0,100)')
    
time.sleep(2)


wait = WebDriverWait(driver, 10)
image_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img.rg_i")))


image_urls = []
for image_element in image_elements:
    source = image_element.get_attribute("src")
    if source is not None:
        if source[0:4]=='http':
            image_urls.append(source)
    time.sleep(2)

for i, image_url in enumerate(image_urls[:10]):
    print('Downloading {0} of 10'.format(i))
    response = requests.get(image_url)
    open("cake_{}.jpg".format(i), "wb").write(response.content)
    
time.sleep(2)

#For storing images in one file
from PIL import Image

# Open the images
images = [Image.open(f"cake_{i}.jpg") for i in range(10)]

# Creating a new PDF file to append the images
pdf_filename = "cakes.pdf"
pdf_file = Image.new("RGB", images[0].size, (255, 255, 255))

# Appending the images to the PDF file
pdf_file.save(pdf_filename, "PDF" ,resolution=100.0, save_all=True, append_images=images)

In [ ]:
driver.close()

#### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped:
1. Brand Name
2. Smartphone name
3. Colour,
4. RAM,
5. “Storage(ROM)”,
6. Primary Camera”,
7. “Secondary Camera”,
8. “Display Size”,
9. “Battery Capacity”,
10. “Price”,
11. “Product URL”.

- Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [11]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.flipkart.com/') 
time.sleep(5)

button = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/button')
button.click()

# Serach for keywords
phones = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
phones.send_keys('Oneplus Nord')    

#Clicking the search button
search = driver.find_element(By.CLASS_NAME,'L0Z3Pu')
search.click()

driver.maximize_window()

In [12]:
product_urls = []

url = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for i in url:
    product_urls.append(i.get_attribute('href'))

In [13]:
Brand = []
Smartphone = []
Display = []
RAM = []
ROM = []
Camera = []
Price = []
Battery_capacity = []
Colour = []

for url in product_urls:
    driver.get(url)
    time.sleep(5)
    
    
    try:
        #For Brand
        name = driver.find_element(By.CLASS_NAME,'B_NuCI')
        Brand.append(name.text.split(' ')[0])   
    except NoSuchElementException:
        pass
    
    try:
        #For Product name
        name = driver.find_element(By.CLASS_NAME,'B_NuCI')
        Smartphone.append(name.text)
    except NoSuchElementException:
        pass        
        
    try:    
        #For display
        dis = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/ul/li[2]')
        Display.append(dis.text)
    except NoSuchElementException:
        pass        
        
    try:
        #For Storage
        Storage = driver.find_element(By.XPATH,'//div[@class="_2418kt"]')
        RAM.append(Storage.text.split('\n')[0].split('|')[0])
        ROM.append(Storage.text.split('\n')[0].split('|')[1])
    except NoSuchElementException:
        pass
    
    
    try:
        price = driver.find_element(By.XPATH,'//div[@class="dyC4hf"]')
        Price.append(price.text.replace('\ni',''))
    except NoSuchElementException:
        pass
    
    try:
        cam = driver.find_element(By.XPATH,'//div[@class="_2418kt"]')
        Camera.append(cam.text.split('\n')[2])
    except NoSuchElementException:
        pass
    
    try:
        #For Battery
        bat = driver.find_element(By.XPATH,'//div[@class="_2418kt"]')
        Battery_capacity.append(bat.text.split('\n')[3])
    except NoSuchElementException:
        pass
    
    try:
        col = driver.find_element(By.XPATH,'(//tr)[4]')
        Colour.append(col.text.replace('Color\n',''))
    except NoSuchElementException:
        pass
    


In [14]:
len(Display),len(Brand),len(Smartphone),len(RAM),len(ROM),len(Battery_capacity),len(Colour),len(Price),len(Camera)

(24, 24, 24, 24, 24, 24, 24, 24, 24)

In [15]:
data = {
    
    'Brand': Brand,
    'Name':Smartphone,
    'RAM':RAM,
    'ROM':ROM,
    'Display':Display,
    'Camera':Camera,
    'Battery Capacity':Battery_capacity,
    'Colour':Colour,
    'Price':Price,
    'Product Link':product_urls
}
df = pd.DataFrame(data)
df

,Brand,Name,RAM,ROM,Display,Camera,Battery Capacity,Colour,Price,Product Link
0,OnePlus,"OnePlus Nord CE 2 Lite (Black Dusk, 128 MB) (...",6 GB RAM,128 MB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Black Dusk,"₹19,884",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Blue Tide,"₹19,934",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Blue Tide,"₹19,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Blue Tide,"₹19,950",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Black Dusk,"₹19,890",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) HD+ Display,64MP Rear Camera | 16MP Front Camera,5000 mAh Battery,Blue Tide,"₹19,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",6 GB RAM,128 GB ROM,16.74 cm (6.59 inch) Display,64MP Rear Camera,5000 mAh Battery,Black Dusk,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,OnePlus,"OnePlus NORD 2T 5G (Gray Shadow, 128 GB) (8 G...",8 GB RAM,128 GB ROM,17.02 cm (6.7 inch) Display,50MP Rear Camera,4500 mAh Battery,Gray Shadow,"₹28,970",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
8,OnePlus,"OnePlus Nord CE 2 5G (Bahama Blue, 128 GB) (8...",8 GB RAM,128 GB ROM,16.33 cm (6.43 inch) Display,64MP Rear Camera,4500 mAh Battery,Bahama Blue,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
9,OnePlus,"OnePlus Nord CE 5G (Charcoal Ink, 128 GB) (6 ...",6 GB RAM,128 GB ROM,16.33 cm (6.43 inch) Display,64MP Rear Camera,4500 mAh Battery,Charcoal Ink,"₹20,899",https://www.flipkart.com/oneplus-nord-ce-5g-ch...


In [16]:
df.to_csv('FLipkartProducts.csv',index=False)
driver.close()

#### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [19]:
City = input('Enter the name of the city: ')

Enter the name of the city: Mumbai


In [20]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.google.co.in/maps/@19.0418832,73.0287206,15z?hl=en-GB') 
time.sleep(2)

city = driver.find_element(By.XPATH,'//input[@class="searchboxinput xiQnY"]').send_keys(City)
search = driver.find_element(By.XPATH,'//*[@id="searchbox-searchbutton"]').click()
time.sleep(2)

try:
    url_string = driver.current_url
    print('URL extracted ',url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    Latitude = [i.split(',')[0] for i in lat_lng]
    Longitude = [i.split(',')[1] for i in lat_lng]
  
except:
    pass

print(Latitude,Longitude)
driver.close()

URL extracted  https://www.google.co.in/maps/place/Mumbai,+Maharashtra/@19.0418832,73.0287206,15z/data=!4m5!3m4!1s0x3be7c6306644edc1:0x5da4ed8f8d648c69!8m2!3d19.0759837!4d72.8776559?hl=en-GB
['19.0418832'] ['73.0287206']


#### 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [22]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.digit.in/') 
time.sleep(2)

Laptops = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[4]/ul/li[3]/a')
Laptops.click()
time.sleep(2)

gaming = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[2]/div[2]/ul/li[10]/a')
gaming.click()

driver.maximize_window()

In [24]:
Card = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[7]/td[3]')
Graphic_card = [i.text for i in Card]


por = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[3]/td[3]')
Processor = [i.text for i in  por]

dis = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[4]/td[3]')
Display = [i.text for i in dis]


os = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[5]/td[3]')
OS = [i.text for i in os]


memo = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[6]/td[3]')
Memory = [i.text for i in memo]

body = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[3]/div[3]/table/tbody/tr[8]/td[3]')
Body = [i.text for i in body]

name = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[1]/div[1]/a/h3')
Name = [i.text for i in name]

In [25]:
product_urls = []

url = driver.find_elements(By.XPATH,'/html/body/div[5]/div[1]/div[3]/div[2]/div/div[1]/div[1]/a')
for i in url:
    product_urls.append(i.get_attribute('href'))

In [27]:
data = {
    
    'Product Name': Name,
    'Processor': Processor,
    'Display': Display,
    'Operating System': OS,
    'Memory' : Memory,
    'Graphics Card':Graphic_card,
    'Body':Body,
    'Product link' : product_urls
}
df = pd.DataFrame(data)
df

,Product Name,Processor,Display,Operating System,Memory,Graphics Card,Body,Product link
0,MSI Titan GT77-12UHS,12th Gen Intel Core i9-12900HX 16 core processor,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,https://www.digit.in/laptops/msi-titan-gt77-12...
1,Alienware X17 R2,12th Gen Intel Core i9-12900H 14 core processo...,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight,https://www.digit.in/laptops/dell-new-alienwar...
2,Acer Predator Triton 500 SE PT516-52s,12th Gen Intel Core i7-12700H 14 core processo...,16″ (2560 x 1600) screen,Windows 11,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,262 x 359 x 20 mm dimension & 2.4 kg weight,https://www.digit.in/laptops/acer-predator-tri...
3,Omen By HP (16-B1371TX),12th Gen Intel Core i7-12700H 14 core processo...,16.1″ (2560 x 1440) screen,Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,https://www.digit.in/laptops/omen-by-hp-16-b13...
4,Acer Predator Helios 300 AN515-45,AMD Ryzen 9-5900HX 8 core processor,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,https://www.digit.in/laptops/acer-nitro-5-an51...
5,MSI Delta 15 (A5EFK-083IN),AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,https://www.digit.in/laptops/msi-delta-15-5th-...
6,Omen By HP (16-C0141AX),AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16.1″ (2560 x 1440) screen,Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,https://www.digit.in/laptops/hp-omen-16-c0141a...
7,Lenovo Legion 5i Pro (82RF00MGIN),12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight,https://www.digit.in/laptops/lenovo-legion-5-p...
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,AMD Ryzen R7-5800H 8 core processor with 4.40 ...,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,https://www.digit.in/laptops/dell-alienware-m1...
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,"15.6″ screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,https://www.digit.in/laptops/lenovo-legion-s7-...


In [28]:
driver.close()

#### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
1. “Rank”,
2. “Name”,
3. “Net worth”,
4. “Age”,
5. “Citizenship”,
6. “Source”,
7. “Industry”.

In [30]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.forbes.com/?sh=3d70ba1c2254') 
time.sleep(2)

dropdown = driver.find_element(By.XPATH,'//div[@class="_69hVhdY4"]')
dropdown.click()
time.sleep(2)

billion = driver.find_element(By.XPATH,'/html[1]/body[1]/div[1]/header[1]/nav[1]/div[1]/div[1]/div[1]/div[2]/ul[1]/li[1]/div[1]/span[1]')
billion.click()
time.sleep(1)

new = driver.find_element(By.XPATH,'/html[1]/body[1]/div[1]/header[1]/nav[1]/div[1]/div[1]/div[1]/div[2]/ul[1]/li[1]/div[2]/div[3]/ul[1]/li[1]/a[1]')
new.click()
time.sleep(1)

driver.maximize_window()
time.sleep(1)
#Using list comprehension and scraping the required data
 
#For Names 
    
name = driver.find_elements(By.XPATH,'//div[@class="personName"]')
Names = [i.text for i in name]
        
# for Rank    
rank = driver.find_elements(By.XPATH,'//div[@class="rank"]')
Ranks = [i.text for i in rank]

#For Net worth
worth = driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
Net_Worth = [i.text for i in worth]

age = driver.find_elements(By.XPATH,'//div[@class="age"]')
Age = [i.text for i in age]

citizen = driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
Citizenship = [i.text for i in citizen]

source = driver.find_elements(By.XPATH,'//span[@class="source-text"]')
Sources = [i.text for i in source]

industry = driver.find_elements(By.XPATH,'//div[@class="category"]')
Industry = [i.text for i in industry]


time.sleep(3)

data = {
    'Rank': Ranks,
    'Name':Names,
    'Net Worth':Net_Worth,
    'Age':Age,
    'CitizenShip':Citizenship,
    'Source':Sources,
    'Industry':Industry   
    
}


In [31]:
df = pd.DataFrame(data)
df

,Rank,Name,Net Worth,Age,CitizenShip,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [32]:
driver.close()

#### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [8]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.youtube.com/') 
time.sleep(2)

driver.maximize_window()

vid = driver.find_element(By.XPATH,'//input[@placeholder="Search"]').send_keys('david attenborough documentary')
search = driver.find_element(By.XPATH,'//*[@id="search-icon-legacy"]').click()
time.sleep(2)

#Clicking the first Video
video = driver.find_element(By.XPATH,'//*[@id="video-title"]/yt-formatted-string').click()

time.sleep(3)

for _ in range(1000):
    driver.execute_script('window.scrollBy(0,1000)')
    

In [14]:
Time = []
Comment = []

# Find all the comment elements
#CommentBox = driver.find_elements(By.XPATH,'//*[@id="main"]')[:500]

data = [i.text for i in CommentBox]
for i in data[:500]:
    Time.append(i.split('\n')[1])
    Comment.append(i.split('\n')[2])

    
vote = driver.find_elements(By.XPATH,'//*[@id="vote-count-middle"]')
Upvotes = [i.text for i in vote[:500]]

In [16]:
data = {
    'Time':Time,
    'Comment':Comment,
    'Comment Upvotes':Upvotes
}
df = pd.DataFrame(data)
df

,Time,Comment,Comment Upvotes
0,2 weeks ago,Thank you Netflix for putting these documentar...,14
1,2 weeks ago (edited),"Everything is connected......you, me , plants ...",8
2,3 months ago,I hope so badly that videos like these produce...,36
3,2 weeks ago,Thanks you Netflix for putting this on you Tub...,3
4,12 days ago,Best document ever!,
...,...,...,...
495,3 weeks ago,Awesome video,
496,1 year ago,Sir David Attenborough needs a podcast talking...,2
497,6 months ago,How do they get those space shots cause wow,
498,2 years ago,Thank you Netflix for putting this on YouTube ...,7


In [17]:
driver.close()

#### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape:
1. hostel name
2. distance from city centre
3. ratings
4. total reviews,
5. overall reviews
6. privates from price
7. dorms from price
8. facilities
9. property description.

In [2]:
#First Connecting with the Web Driver
driver = webdriver.Chrome(r'C:\Users\chizz\Downloads\chromedriver_win32.zip\chromedriver.exe')
driver.get('https://www.hostelworld.com/') 
time.sleep(2)

driver.maximize_window()
time.sleep(1)

In [7]:
# Selecitng the London Element(Location)
try:
    search = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
except NoSuchElementException:
    search = driver.find_element(By.XPATH,'//input[@class="search-input"]')
    
    
search.send_keys('London')    
time.sleep(3)

London_tag = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
London_tag.click()

Lets_go = driver.find_element(By.XPATH,'//*[@id="search-button"]')
Lets_go.click()

In [55]:
Hostel_name = []
Distance = []
Rating = []
Demo_Price = []


while True:
    name = driver.find_elements(By.XPATH,'//div[@class="property-name"]')
    distance = driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
    rating = driver.find_elements(By.XPATH,'//div[@class="property-rating"]')
    price = driver.find_elements(By.XPATH,'//div[@class="property-accommodations"]')
    time.sleep(3)
    
    for i in name:
        Hostel_name.append(i.text)
    
    for d in distance:
        Distance.append(d.text.split('-')[1])
    
    for r in rating:
        Rating.append(r.text) 
        
    for p in price:
        Demo_Price.append(p.text)        
    
    try:
        next_button = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div/div/div[3]/div/button[2]')    
        try:
            next_button.click()     
        except ElementClickInterceptedException:
            pass
    except NoSuchElementException:
        break

In [ ]:
Hostel_name = Hostel_name[34:]
Page_one_hostel = Hostel_name[2:32]
Page_two_hostel = Hostel_name[-18:]
Hostel_name = Page_one_hostel+Page_two_hostel

In [84]:
Page_one_Distance = Distance[:30]
Page_two_Distance = Distance[-18:]
Distance = Page_one_Distance+Page_two_Distance

In [101]:
Rating_1 = Rating[2:32]
Rating_2 = Rating[-17:]
Rating = Rating_1+Rating_2
Rating.insert(-1,'-')
Rating.remove('-')

In [123]:
Page1 = Demo_Price[2:32]
Page2 = Demo_Price[-18:]
Demo_Price = Page1 + Page2

In [132]:
Final_Price = []

for i in Demo_Price:
    Final_Price.append(i.split('\n'))

In [134]:
Privates = []
Dorms = []


for i in Final_Price:
    if 'Privates From' in i:
        Privates.append(i[0:2])
        
    else:
        Privates.append(i[0])
        
        
for i in Final_Price:
    
    if 'Dorms From' in i:
        Dorms.append(i[-1])
        
    else:
        Dorms.append(i[-1])


In [141]:
New_Ratings = []

for i in Rating:
    New_Ratings.append(i.split('\n'))

In [151]:
Total_reviews = []
Overall = []
RATING = []

for i in New_Ratings:
    RATING.append(i[0])
    if len(i)==3:
        Overall.append(i[1])
        Total_reviews.append(i[2])
    elif len(i)==2:
        Overall.append('-')
        Total_reviews.append(i[1])
    else:
        Overall.append('-')
        Total_reviews.append('-')

        
        
    

In [ ]:
Privates = []
Dorms = []

for i in Prices:
    if 'Privates From' in i:
        Privates.append(i[1])
        
    else:
        Privates.append(i[0])
        
        
for i in Prices:
    
    if 'Dorms From' in i:
        Dorms.append(i[-1])
        
    else:
        Dorms.append(i[2])

In [ ]:
Facilities = []
Description = []

for i in URLs:
    driver.get(i)
    time.sleep(1)
    faci = driver.find_element(By.CLASS_NAME,'facility-sections')
    Facilities.append(faci.text.replace('\n','//'))   
    
    des = driver.find_element(By.XPATH,'//div[@class="content collapse-content"]')
    Description.append(des.text)

In [156]:
data = {
    'Hostel Name':Hostel_name,
    'Distance from City Centre':Distance,
    'Ratings':RATING,
    'Total Reviews':Total_reviews,
    'Overall Reviews':Overall,
    'Privates from Price':Privates,
    'Dorms from Price':Dorms,
    
}

In [157]:
df = pd.DataFrame(data)
df

,Hostel Name,Distance from City Centre,Ratings,Total Reviews,Overall Reviews,Privates from Price,Dorms from Price
0,Wombat's City Hostel London,3.6km from city centre,8.9,(14424),Fabulous,No Privates Available,"₹4,068"
1,St Christopher's Village,1.8km from city centre,8.0,(11776),Fabulous,No Privates Available,"₹1,752"
2,Hostel One Notting Hill,5.5km from city centre,9.3,(1646),Superb,"[Privates From, ₹16,089]","₹5,050"
3,Urbany Hostel London,5.4km from city centre,9.2,(580),Superb,"[Privates From, ₹16,501]","₹3,324"
4,Generator London,3km from city centre,7.7,(7297),Very Good,"[Privates From, ₹7,936]","₹2,581"
5,Safestay London Elephant & Castle,1.7km from city centre,7.3,(4738),Very Good,"[Privates From, ₹6,439]","₹1,437"
6,Smart Russell Square Hostel,2.6km from city centre,7.0,(9848),Very Good,No Privates Available,"₹2,303"
7,St Christopher's Camden,4.3km from city centre,8.2,(3934),Fabulous,No Privates Available,"₹2,084"
8,No.8 Seven Sisters,9km from city centre,5.5,(3930),-,No Privates Available,"₹1,160"
9,Safestay London Kensington Holland Park,5.9km from city centre,7.0,(1402),Very Good,No Privates Available,"₹1,445"


In [158]:
driver.close()